In [ ]:
!pip install memory-profiler

In [ ]:
# For runtime and memory profiling
from memory_profiler import memory_usage
import time
from collections import defaultdict

# For finding frequen patterns
from collections import defaultdict, Counter, OrderedDict
from typing import Dict, List, Tuple, Iterable, Optional, Set, FrozenSet
from itertools import combinations


In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Updated Code

In [ ]:

#Updated Code

class FPNode:
    __slots__ = ("item", "count", "parent", "children", "node_link")
    def __init__(self, item=None, count=0, parent=None):
        self.item = item
        self.count = count
        self.parent = parent
        self.children = {}
        self.node_link = None

    def increment(self, n=1):
        self.count += n

class FPTree:
    def __init__(self):
        self.root = FPNode()
        self.header: Dict[str, Tuple[int, Optional[FPNode]]] = {}

    def add_transaction(self, items: List[str], count: int = 1):
        node = self.root
        for item in items:
            if item not in node.children:
                child = FPNode(item=item, parent=node)
                node.children[item] = child
                supp, head = self.header.get(item, (0, None))
                if head is None:
                    self.header[item] = (supp, child)
                else:
                    cur = head
                    while cur.node_link:
                        cur = cur.node_link
                    cur.node_link = child
            node = node.children[item]
            node.count += count
        # update header supports
        for item in items:
            supp, head = self.header[item]
            self.header[item] = (supp + count, head)

    def items_by_ascending_support(self) -> List[str]:
        return sorted(self.header.keys(), key=lambda i: self.header[i][0])

    def single_path(self) -> bool:
        node = self.root
        while node:
            if len(node.children) > 1:
                return False
            node = next(iter(node.children.values()), None)
        return True

def build_fp_tree(transactions: Iterable[List[str]], minsup: int) -> Optional[FPTree]:
    freq_items = defaultdict(int)
    for t in transactions:
        for item in t:
            freq_items[item] += 1
    freq_items = {i: c for i, c in freq_items.items() if c >= minsup}
    if not freq_items:
        return None
    # descending order for prefix-sharing
    order_key = sorted(freq_items.items(), key=lambda x: (-x[1], x[0]))
    header_table = OrderedDict()
    for item, count in order_key:
        header_table[item] = [count, None]
    tree = FPTree()
    for t in transactions:
        filtered = [i for i in t if i in freq_items]
        filtered.sort(key=lambda x: next(idx for idx, (itm, _) in enumerate(order_key) if itm == x))
        tree.add_transaction(filtered)
    return tree

def ascend_path(node: FPNode) -> List[str]:
    path = []
    cur = node.parent
    while cur and cur.item is not None:
        path.append(cur.item)
        cur = cur.parent
    return path

def fp_growth(tree: FPTree, minsup: int, suffix: Tuple[str, ...], out: Dict[FrozenSet[str], int]):
    # Mine items without conditional FP-trees
    for item in tree.items_by_ascending_support():
        pattern = tuple(sorted((item,) + suffix))
        supp = tree.header[item][0]
        out[frozenset(pattern)] = supp

        # collect all prefix paths
        _, head = tree.header[item]
        paths = []
        cur = head
        while cur:
            p = ascend_path(cur)
            if p:
                paths.append((p, cur.count))
            cur = cur.node_link

        # count support for combinations of prefix items
        counts = defaultdict(int)
        for path, c in paths:
            for r in range(1, len(path) + 1):
                for comb in combinations(path, r):
                    counts[frozenset(comb)] += c
        # add each combination with the current item
        for comb_set, c in counts.items():
          out[frozenset(sorted(set(comb_set).union({item})))] = c


def mine_fp_growth(transactions: Iterable[Iterable[str]], minsup: int) -> Dict[FrozenSet[str], int]:
    tree = build_fp_tree(transactions, minsup)
    out: Dict[FrozenSet[str], int] = {}
    if tree:
        fp_growth(tree, minsup, tuple(), out)
    return out

In [ ]:
# Main program
if __name__ == "__main__":
    # Example from many FP-growth tutorials. You can use it to verify your program.
    # transactions = [
    #     ["f", "a", "c", "d", "g", "i", "m", "p"],
    #     ["a", "b", "c", "f", "l", "m", "o"],
    #     ["b", "f", "h", "j", "o"],
    #     ["b", "c", "k", "s", "p"],
    #     ["a", "f", "c", "e", "l", "p", "m", "n"]
    # ]

    minsup = 0.01  # Minimum Support Threshold

    # Read a transaction file in Google Drive
    # Replace "4412/2025F/A1/Datasets/walmart1.csv" with your file path
    file_path = "/content/bakery.csv"
    # file_path = "/content/walmart1.csv"

    # Read transactions from the file
    transactions = []
    try:
        with open(file_path, 'r') as f:
            for line in f:
                # Assuming each line is a comma-separated list of item ids
                transaction = line.strip().split(',')
                if transaction: # Avoid adding empty transactions
                    transactions.append(transaction)
    except FileNotFoundError:
        print(f"Error: File not found at {file_path}")
        transactions = [] # Empty transactions list if file not found
    except Exception as e:
        print(f"An error occurred while reading the file: {e}")
        transactions = [] # Empty transactions list if an error occurs

    minsup_count=minsup*len(transactions)
    print(f"Number of transactions: {len(transactions)}")
    print(f"Minimum support count: {minsup_count}")

    if not transactions:
        print("No transactions loaded. Please check the file path and content.")
    else:
        # Measure runtime and memory usage and call mine_fp_growth
        start_time = time.time()
        mem_usage, patterns = memory_usage((mine_fp_growth, (transactions, minsup_count)), interval=0.1, retval=True)
        end_time = time.time()

        print("== Updated FP-growth (No conditional Tree)==")
        for pat, sup in sorted(patterns.items(), key=lambda x: (len(x[0]), sorted(list(x[0])), x[1])):
            print(set(pat), ":", sup)

        # Count frequent itemsets by length
        itemset_counts_by_length = defaultdict(int)
        for itemset in patterns:
            itemset_counts_by_length[len(itemset)] += 1

        print("\n== Frequent Itemset Counts by Length ==")
        for length, count in sorted(itemset_counts_by_length.items()):
            print(f"Length {length}: {count}")

        # Total number of frequent itemsets
        total_itemsets = len(patterns)
        print(f"\nTotal number of frequent itemsets: {total_itemsets}")

        # Print runtime and memory usage
        print(f"\nRuntime: {end_time - start_time:.4f} seconds")
        print(f"Peak memory usage: {max(mem_usage):.4f} MiB")


Streaming output truncated to the last 5000 lines.
{'20', '9', '13', '45'} : 1
{'24', '39', '21', '13'} : 1
{'24', '41', '21', '13'} : 1
{'24', '45', '21', '13'} : 1
{'24', '9', '21', '13'} : 1
{'41', '39', '21', '13'} : 1
{'39', '21', '13', '45'} : 1
{'39', '21', '13', '9'} : 1
{'9', '21', '13', '4'} : 1
{'41', '45', '21', '13'} : 1
{'41', '21', '13', '47'} : 1
{'41', '48', '21', '13'} : 1
{'41', '9', '21', '13'} : 1
{'45', '21', '13', '9'} : 1
{'48', '21', '13', '47'} : 1
{'5', '22', '13', '30'} : 1
{'45', '22', '13', '42'} : 1
{'9', '22', '13', '42'} : 1
{'45', '22', '13', '9'} : 1
{'24', '41', '23', '13'} : 1
{'23', '13', '33', '25'} : 1
{'23', '13', '40', '25'} : 1
{'9', '23', '13', '25'} : 1
{'23', '13', '33', '40'} : 1
{'9', '23', '13', '33'} : 1
{'9', '23', '13', '40'} : 1
{'24', '35', '13', '26'} : 1
{'24', '39', '13', '26'} : 1
{'24', '13', '26', '47'} : 1
{'24', '35', '39', '13'} : 1
{'24', '35', '13', '47'} : 1
{'24', '36', '13', '46'} : 1
{'24', '36', '13', '47'} : 1
{'24'

In [ ]:
#old Code
class FPNode:
    __slots__ = ("item", "count", "parent", "children", "node_link")
    def __init__(self, item=None, count=0, parent=None):
        self.item = item
        self.count = count              # exactly one count per node
        self.parent = parent
        self.children: Dict[str, "FPNode"] = {}
        self.node_link: Optional["FPNode"] = None  # next node of same item

    def is_root(self) -> bool:
        return self.item is None

class FPTree:
    def __init__(self):
        self.root = FPNode()
        # header: item -> (support, head_of_node_link_chain)
        self.header: Dict[str, Tuple[int, Optional[FPNode]]] = {}

    def add_transaction(self, items: List[str], count: int = 1):
        node = self.root
        for item in items:
            if item not in node.children:
                child = FPNode(item=item, count=0, parent=node)
                node.children[item] = child
                # link into header
                supp, head = self.header.get(item, (0, None))
                if head is None:
                    self.header[item] = (supp, child)
                else:
                    # append child to the end of node-link chain
                    cur = head
                    while cur.node_link is not None:
                        cur = cur.node_link
                    cur.node_link = child
                # keep support untouched here; update after loop
            node = node.children[item]
            node.count += count
        # update header supports
        for item in items:
            supp, head = self.header[item]
            self.header[item] = (supp + count, head)

    def items_by_ascending_support(self) -> List[str]:
        return sorted(self.header.keys(), key=lambda i: self.header[i][0])

    def single_path(self) -> bool:
        # true if every node has at most one child
        def has_multiple_children(n: FPNode) -> bool:
            if len(n.children) > 1:
                return True
            for c in n.children.values():
                if has_multiple_children(c):
                    return True
            return False
        return not has_multiple_children(self.root)

def build_fp_tree(transactions: Iterable[Iterable[str]], minsup: int) -> FPTree:
    # 1) count item supports
    flat = Counter()
    for t in transactions:
        flat.update(set(t))  # set: standard FP-growth counts each item once per trans
    # 2) filter infrequent items
    freq_items = {i for i, c in flat.items() if c >= minsup}
    if not freq_items:
        return FPTree()
    # 3) order by descending support, tie-break lexicographically for determinism
    order_key = {i: (-flat[i], i) for i in freq_items}

    # 4) build tree
    tree = FPTree()
    for t in transactions:
        filtered = [i for i in t if i in freq_items]
        if not filtered:
            continue
        filtered.sort(key=lambda x: order_key[x])
        tree.add_transaction(filtered, count=1)
    return tree

def ascend_path(node: FPNode) -> List[str]:
    path = []
    cur = node.parent
    while cur is not None and not cur.is_root():
        path.append(cur.item)
        cur = cur.parent
    return path

def conditional_pattern_base(tree: FPTree, item: str) -> List[Tuple[List[str], int]]:
    # collect all prefix paths leading to 'item' nodes
    patterns = []
    _, head = tree.header[item]
    cur = head
    while cur is not None:
        path = ascend_path(cur)
        if path:
            patterns.append((path, cur.count))  # multiplicity = node count
        cur = cur.node_link
    return patterns

def fp_growth(tree: FPTree, minsup: int, suffix: Tuple[str, ...], out: Dict[FrozenSet[str], int]):
    # mine items in ascending support order (standard FP-growth choice)
    for item in tree.items_by_ascending_support():
        # pattern = item ∪ suffix
        supp = tree.header[item][0]
        pattern = tuple(sorted((item,) + suffix))
        out[frozenset(pattern)] = supp

        # build conditional FP-tree for this item
        cpb = conditional_pattern_base(tree, item)
        if not cpb:
            continue

        # build conditional transactions with multiplicities
        cond_tx = []
        for path, w in cpb:
            # filter infrequent in conditional db will be handled by build_fp_tree
            cond_tx.extend([path] * w)

        cond_tree = build_fp_tree(cond_tx, minsup)
        if not cond_tree.header:
            continue

        # single path optimization
        if cond_tree.single_path():
            # enumerate all combinations along the single path
            # collect the single path items (descending order in the path)
            path_items = []
            node = cond_tree.root
            while node.children:
                (child_item, child_node), = node.children.items()
                path_items.append((child_item, child_node.count))
                node = child_node

            # generate all non-empty subsets
            items = [i for i, _ in path_items]
            # The support of a combination is min count along its nodes
            from itertools import combinations
            for r in range(1, len(items) + 1):
                for combo in combinations(items, r):
                    # min support among nodes in combo (found in the conditional tree)
                    # We can recompute by walking again or track counts by item map
                    # For simplicity, rebuild a map:
                    counts = {}
                    for i, c in path_items:
                        counts[i] = c
                    supp_combo = min(counts[i] for i in combo)
                    out[frozenset(tuple(sorted(combo + pattern)))] = supp_combo
        else:
            fp_growth(cond_tree, minsup, (item,) + suffix, out)

def mine_fp_growth(transactions: Iterable[Iterable[str]], minsup: int) -> Dict[FrozenSet[str], int]:
    tree = build_fp_tree(transactions, minsup)
    out: Dict[FrozenSet[str], int] = {}
    if tree.header:
        fp_growth(tree, minsup, suffix=tuple(), out=out)
    return out



In [ ]:
# Main program
if __name__ == "__main__":
    # Example from many FP-growth tutorials. You can use it to verify your program.
    # transactions = [
    #     ["f", "a", "c", "d", "g", "i", "m", "p"],
    #     ["a", "b", "c", "f", "l", "m", "o"],
    #     ["b", "f", "h", "j", "o"],
    #     ["b", "c", "k", "s", "p"],
    #     ["a", "f", "c", "e", "l", "p", "m", "n"]
    # ]

    minsup = 0.01  # Minimum Support Threshold

    # Read a transaction file in Google Drive
    # Replace "4412/2025F/A1/Datasets/walmart1.csv" with your file path
    file_path = "/content/bakery.csv"
    # file_path = "/content/walmart1.csv"

    # Read transactions from the file
    transactions = []
    try:
        with open(file_path, 'r') as f:
            for line in f:
                # Assuming each line is a comma-separated list of item ids
                transaction = line.strip().split(',')
                if transaction: # Avoid adding empty transactions
                    transactions.append(transaction)
    except FileNotFoundError:
        print(f"Error: File not found at {file_path}")
        transactions = [] # Empty transactions list if file not found
    except Exception as e:
        print(f"An error occurred while reading the file: {e}")
        transactions = [] # Empty transactions list if an error occurs

    minsup_count=minsup*len(transactions)
    print(f"Number of transactions: {len(transactions)}")
    print(f"Minimum support count: {minsup_count}")

    if not transactions:
        print("No transactions loaded. Please check the file path and content.")
    else:
        # Measure runtime and memory usage and call mine_fp_growth
        start_time = time.time()
        mem_usage, patterns = memory_usage((mine_fp_growth, (transactions, minsup_count)), interval=0.1, retval=True)
        end_time = time.time()

        print("== Original FP-growth ==")
        for pat, sup in sorted(patterns.items(), key=lambda x: (len(x[0]), sorted(list(x[0])), x[1])):
            print(set(pat), ":", sup)

        # Count frequent itemsets by length
        itemset_counts_by_length = defaultdict(int)
        for itemset in patterns:
            itemset_counts_by_length[len(itemset)] += 1

        print("\n== Frequent Itemset Counts by Length ==")
        for length, count in sorted(itemset_counts_by_length.items()):
            print(f"Length {length}: {count}")

        # Total number of frequent itemsets
        total_itemsets = len(patterns)
        print(f"\nTotal number of frequent itemsets: {total_itemsets}")

        # Print runtime and memory usage
        print(f"\nRuntime: {end_time - start_time:.4f} seconds")
        print(f"Peak memory usage: {max(mem_usage):.4f} MiB")


Number of transactions: 1000
Minimum support count: 10.0
== Original FP-growth ==
{'0'} : 84
{'1'} : 85
{'10'} : 41
{'11'} : 68
{'12'} : 79
{'13'} : 56
{'14'} : 95
{'15'} : 73
{'16'} : 81
{'17'} : 51
{'18'} : 84
{'19'} : 76
{'2'} : 72
{'20'} : 40
{'21'} : 44
{'22'} : 108
{'23'} : 82
{'24'} : 66
{'25'} : 38
{'26'} : 47
{'27'} : 90
{'28'} : 102
{'29'} : 61
{'3'} : 78
{'30'} : 49
{'31'} : 91
{'32'} : 76
{'33'} : 78
{'34'} : 42
{'35'} : 75
{'36'} : 84
{'37'} : 65
{'38'} : 26
{'39'} : 55
{'4'} : 91
{'40'} : 66
{'41'} : 72
{'42'} : 82
{'43'} : 62
{'44'} : 77
{'45'} : 94
{'46'} : 85
{'47'} : 74
{'48'} : 77
{'49'} : 59
{'5'} : 103
{'6'} : 34
{'7'} : 93
{'8'} : 37
{'9'} : 90
{'0', '2'} : 40
{'0', '46'} : 47
{'19', '1'} : 40
{'11', '37'} : 29
{'11', '45'} : 29
{'11', '7'} : 33
{'31', '12'} : 44
{'36', '12'} : 41
{'48', '12'} : 36
{'9', '13'} : 10
{'14', '22'} : 12
{'14', '44'} : 34
{'14', '5'} : 13
{'49', '15'} : 24
{'15', '7'} : 31
{'32', '16'} : 40
{'45', '16'} : 33
{'29', '17'} : 18
{'17', '4